In [1]:
import numpy as np

In [15]:
import numpy as np

# Your provided pitch feature coordinates
bottom_left_12_box_point = np.array([319, 672, 69, 39])  # Format: x, y, width, height

# Your provided homography matrix
homography_matrix = np.array([
    [0.34523, 0.24001, 511.79],
    [0.26268, 0.75285, -484.6],
    [0.0029172, 0.028532, 1]
])

# Function to transform a point using homography
def transform_point_using_homography(homography_matrix, image_point_xywh):
    """
    Transform a point from image coordinates to world coordinates using a homography matrix.
    
    Args:
        homography_matrix: 3x3 homography matrix
        image_point_xywh: Point in [x, y, width, height] format
        
    Returns:
        Transformed point in world coordinates (X, Y)
    """
    # Extract the center of the bounding box
    x, y, w, h = image_point_xywh
    
    # Option 1: Center of bounding box
    center_x = x + w / 2
    center_y = y + h / 2
    
    # Option 2: Bottom center (feet position)
    bottom_center_x = x + w / 2
    bottom_center_y = y + h
    
    # Try both center and bottom center for comparison
    points_to_try = [
        ("Center", center_x, center_y),
        ("Bottom Center", bottom_center_x, bottom_center_y)
    ]
    
    results = []
    for point_name, point_x, point_y in points_to_try:
        # Create homogeneous coordinates [x, y, 1]
        homogeneous_point = np.array([point_x, point_y, 1])
        
        # Apply the homography transformation
        transformed_point = np.dot(homography_matrix, homogeneous_point)
        
        # Convert back to 2D coordinates by dividing by the third component
        world_x = transformed_point[0] / transformed_point[2]
        world_y = transformed_point[1] / transformed_point[2]
        
        results.append((point_name, world_x, world_y))
    
    return results

# Perform the transformation and verify results
transformation_results = transform_point_using_homography(homography_matrix, bottom_left_12_box_point)

print("Verification of Homography Transformation")
print("-----------------------------------------")
print(f"Image Point (x,y,w,h): {bottom_left_12_box_point}")
print("Homography Matrix:")
for row in homography_matrix:
    print(f"  {row}")
print("\nTransformation Results:")

for point_name, world_x, world_y in transformation_results:
    print(f"{point_name} Point:")
    print(f"  World Coordinates: X = {world_x:.2f}, Y = {world_y:.2f}")

# For comparison with expected values (if you know them)
# Assuming this point might be the left goal post which should be around (41.34, 0)
# in your world coordinate system
expected_world_point = (35.85, 5.5)  # Based on your world_points list
print("\nExpected Coordinates (if this is the Left Goal Post):")
print(f"  X = {expected_world_point[0]}, Y = {expected_world_point[1]}")

# Calculate error if this is indeed the left goal post
for point_name, world_x, world_y in transformation_results:
    error_x = abs(world_x - expected_world_point[0])
    error_y = abs(world_y - expected_world_point[1])
    print(f"\nError using {point_name} Point:")
    print(f"  X error: {error_x:.2f} meters")
    print(f"  Y error: {error_y:.2f} meters")
    print(f"  Total error: {np.sqrt(error_x**2 + error_y**2):.2f} meters")

Verification of Homography Transformation
-----------------------------------------
Image Point (x,y,w,h): [319 672  69  39]
Homography Matrix:
  [3.4523e-01 2.4001e-01 5.1179e+02]
  [ 2.6268e-01  7.5285e-01 -4.8460e+02]
  [0.0029172 0.028532  1.       ]

Transformation Results:
Center Point:
  World Coordinates: X = 36.75, Y = 5.92
Bottom Center Point:
  World Coordinates: X = 36.05, Y = 6.43

Expected Coordinates (if this is the Left Goal Post):
  X = 35.85, Y = 5.5

Error using Center Point:
  X error: 0.90 meters
  Y error: 0.42 meters
  Total error: 1.00 meters

Error using Bottom Center Point:
  X error: 0.20 meters
  Y error: 0.93 meters
  Total error: 0.95 meters
